# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [9]:
# imports
import panel as pn
from panel.interact import interact
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [2]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_TOKEN")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [3]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
geo_neighborhood = pd.read_csv(file_path)
geo_neighborhood.columns = ['nieghborhood', 'lat','lon']

In [4]:
display(sfo_data.head())

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182945,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


In [5]:
display(geo_neighborhood.head())

,nieghborhood,lat,lon
0,Alamo Square,37.791012,-122.402100
1,Anza Vista,37.779598,-122.443451
2,Bayview,37.734670,-122.401060
3,Bayview Heights,37.728740,-122.410980
4,Bernal Heights,37.728630,-122.443050


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [6]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    housing_units = sfo_data.groupby('year').mean().copy()
    fig = plt.figure()
    ax = housing_units.reset_index().hvplot.bar(
        x='year',
        y='housing_units',
        ylim=[370000, 387500],
        title='Housing Units in San Franciso from 2010 to 2016',
        )
    return ax


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    rent_data = sfo_data.loc[:, ['sale_price_sqr_foot', 'gross_rent']].groupby(['year']).mean().copy()
    fig = plt.figure()
    ax = rent_data['gross_rent'].hvplot.line(
        title='Average Gross Rent in San Francisco Per Year',
    )
    return ax


def average_sales_price():
    """Average Sales Price Per Year."""
    rent_data = sfo_data.loc[:, ['sale_price_sqr_foot', 'gross_rent']].groupby(['year']).mean().copy()
    fig = plt.figure()
    ax = rent_data['sale_price_sqr_foot'].hvplot.line(
        title='Average Sale Price per Square foot in San Francisco Per Year',
    )
    return ax


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    neighborhoods = sfo_data.neighborhood.unique()
    neighborhood_data = sfo_data.groupby(['neighborhood', 'year']).mean().copy()
    
    def interactive_neighborhood(neighborhood):
        ax = neighborhood_data.loc[neighborhood].hvplot.line(
            x='year',
            y='sale_price_sqr_foot'
        )
        return ax
    
    return interact(interactive_neighborhood, neighborhood=neighborhoods)


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    expensive_neighborhoods = sfo_data.groupby('neighborhood').mean().sort_values('sale_price_sqr_foot', ascending=False).head(10)
    fig = plt.figure()
    ax = expensive_neighborhoods['sale_price_sqr_foot'].hvplot.bar(
        title='Top 10 Expensive Neighborhoods in SFO',
        rot=90,
    )
    return ax
    

def parallel_coordinates():
    """Parallel Coordinates Plot."""
    expensive_neighborhoods = sfo_data.groupby('neighborhood').mean().sort_values('sale_price_sqr_foot', ascending=False).head(10)
    tmp_df = expensive_neighborhoods.reset_index().copy()
    ax = px.parallel_coordinates(
        tmp_df,
        dimensions=tmp_df.columns,
        color=tmp_df.index,
        color_continuous_scale=px.colors.sequential.Inferno
    )
    return ax

def parallel_categories():
    """Parallel Categories Plot."""
    expensive_neighborhoods = sfo_data.groupby('neighborhood').mean().sort_values('sale_price_sqr_foot', ascending=False).head(10)
    tmp_df = expensive_neighborhoods.reset_index().copy()
    ax = px.parallel_categories(
        tmp_df,
        dimensions=tmp_df.columns,
        color=tmp_df.index,
        color_continuous_scale=px.colors.sequential.Inferno
    )
    return ax


def neighborhood_map():
    """Neighborhood Map"""
    mean_sfo_data = sfo_data.groupby('neighborhood').mean().copy()
    geo_sfo_data = geo_neighborhood.join(
                    other=mean_sfo_data,
                    on='nieghborhood'   
    ).dropna()
    ax = px.scatter_mapbox(
        geo_sfo_data,
        lat="lat",
        lon="lon",
        size="sale_price_sqr_foot",
        color="gross_rent",
        zoom=10
    )
    return ax

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [7]:
blurb = 'This dashboard presents Rental and Purchasing Statistics for Downtown Sanfrancisco from 2010 to 2016.'

welcome_column = pn.Column(
    blurb,
    neighborhood_map()
)

market_analysis_column = pn.Column(
    '## General Stats',
    housing_units_per_year(),
    average_gross_rent(),
    average_sales_price()
)

neighborhood_column = pn.Column(
    '## Neighborhood Stats',
    average_price_by_neighborhood(),
    top_most_expensive_neighborhoods()
)

parallel_column = pn.Column(
    '## Parallel Coordinate and Category Plots',
    parallel_coordinates(),
    parallel_categories()
)

panel = pn.Tabs(
    ('Welcome'                , welcome_column),
    ('Yearly Market Analysis' , market_analysis_column),
    ('Neighborhood Analysis'  , neighborhood_column),
    ('Parallel Plots Analysis', parallel_column)
)

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

## Serve the Panel Dashboard

In [8]:
panel.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Curve)
        [3] HoloViews(Curve)
    [2] Column
        [0] Markdown(str)
        [1] Column
            [0] Column
                [0] Select(name='neighborhood', options=['Alamo Square', ...], value='Alamo Square')
            [1] Row
                [0] HoloViews(Curve, name='interactive01708')
        [2] HoloViews(Bars)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)